<a href="https://colab.research.google.com/github/avidday/Colab/blob/main/nvcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10203157230596647368
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14328594432
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3927041756139502375
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [2]:
!ls /usr/local/cuda/
!nvcc --version

bin	compute-sanitizer  extras  include  nvml  share  targets
compat	doc		   gds	   lib64    nvvm  src
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [26]:
%%writefile p4x4det.cu
#include <iostream>
#include <vector>
#include <algorithm>

#include <cub/block/block_load.cuh>
#include <cub/block/block_store.cuh>
#include <cub/block/block_exchange.cuh>
#include <cuda/std/complex>

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

const int _4x4 = 16;
const int _5x5 = 25;
const int blocksz = 32;

template<typename T>
__device__ __forceinline__ T det3x3(T a, T b, T c, T d, T e, T f, T g, T h, T i)
{
    auto mul = [&] (T x, T y)      { return x * y; };
    auto fma = [&] (T x, T y, T z) { return mul(x, y) + z; };

    T fh = mul(-f, h);
    T fg = mul(-f, g);
    T eg = mul(-e, g);

    T det_a = fma(e, i, fh);
    T det_b = fma(d, i, fg);
    T det_c = fma(d, h, eg);

    T det = mul( a, det_a);
    det =   fma(-b, det_b, det);
    det =   fma( c, det_c, det);

    return det;
}

template<typename T>
__global__ void det5x5kernel(T *d_data, T *d_result, int Nmatrix)
{

    typedef cub::BlockLoad<T, blocksz, _5x5, cub::BLOCK_LOAD_STRIPED> BlockLoad;
    typedef cub::BlockStore<T, blocksz, 1, cub::BLOCK_STORE_DIRECT> BlockStore;
    typedef cub::BlockExchange<T, blocksz, _5x5> BlockExchange;

    __shared__ 
    union {
	    typename BlockLoad::TempStorage load;
      typename BlockStore::TempStorage store;
      typename BlockExchange::TempStorage exchange;
    } temp;

    T thread_data[_5x5], l_result[1];
    T det;

    auto mul =  [&] (T x, T y)      { return x * y; };
    auto fma =  [&] (T x, T y, T z) { return mul(x, y) + z; };

    BlockLoad(temp.load).Load(d_data, thread_data, Nmatrix * _5x5);
    __syncthreads();

    BlockExchange(temp.exchange).StripedToBlocked(thread_data);

    T a11 = thread_data[0];      T a21 = thread_data[5+0];
    T a12 = thread_data[1];      T a22 = thread_data[5+1];
    T a13 = thread_data[2];      T a23 = thread_data[5+2];
    T a14 = thread_data[3];      T a24 = thread_data[5+3];
    T a15 = thread_data[4];      T a25 = thread_data[5+4];

    T a31 = thread_data[10+0];    T a41 = thread_data[15+0];
    T a32 = thread_data[10+1];    T a42 = thread_data[15+1];
    T a33 = thread_data[10+2];    T a43 = thread_data[15+2];
    T a34 = thread_data[10+3];    T a44 = thread_data[15+3];
    T a35 = thread_data[10+4];    T a45 = thread_data[15+4];

    T a51 = thread_data[20+0];
    T a52 = thread_data[20+1];
    T a53 = thread_data[20+2];
    T a54 = thread_data[20+3];
    T a55 = thread_data[20+4];

    T a11_a22 = mul(a11, a22);
    T a11_a32 = mul(a11, a32);
    T a11_a42 = mul(a11, a42);
    T a11_a52 = mul(a11, a52); 

    T a21_a12 = mul(a21, a12);
    T a21_a32 = mul(a21, a32);
    T a21_a42 = mul(a21, a42);
    T a21_a52 = mul(a21, a52); 



    T det_13_24_35 = det3x3<T>(a13, a14, a15, a23, a24, a25, a33, a34, a35);
    T det_13_24_45 = det3x3<T>(a13, a14, a15, a23, a24, a25, a43, a44, a45);
    T det_13_24_55 = det3x3<T>(a13, a14, a15, a23, a24, a25, a53, a54, a55);
    T det_13_34_45 = det3x3<T>(a13, a14, a15, a33, a34, a35, a43, a44, a45);
    T det_13_34_55 = det3x3<T>(a13, a14, a15, a33, a34, a35, a53, a54, a55);
    T det_13_44_55 = det3x3<T>(a13, a14, a15, a43, a44, a45, a53, a54, a55);
    T det_23_34_45 = det3x3<T>(a23, a24, a25, a33, a34, a35, a43, a44, a45);
    T det_23_34_55 = det3x3<T>(a23, a24, a25, a33, a34, a35, a53, a54, a55);
    T det_23_44_55 = det3x3<T>(a23, a24, a25, a43, a44, a45, a53, a54, a55);
    T det_33_44_55 = det3x3<T>(a33, a34, a35, a43, a44, a45, a53, a54, a55);

    det = mul( a11_a22, det_33_44_55);
    det = fma(-a11_a32, det_23_44_55, det);
    det = fma( a11_a42, det_23_34_55, det);
    det = fma(-a11_a52, det_23_34_55, det); 

    l_result[0] = det;
    BlockStore(temp.store).Store(d_result, l_result, Nmatrix); 

}

template __global__ void det5x5kernel<float>(float *, float *, int);

template<typename T>
__global__ void det4x4kernel(T *d_data, T *d_result, int Nmatrix)
{

    typedef cub::BlockLoad<T, blocksz, _4x4, cub::BLOCK_LOAD_STRIPED> BlockLoad;
    typedef cub::BlockStore<T, blocksz, 1, cub::BLOCK_STORE_DIRECT> BlockStore;
    typedef cub::BlockExchange<T, blocksz, _4x4> BlockExchange;

    __shared__ 
    union {
	    typename BlockLoad::TempStorage load;
      typename BlockStore::TempStorage store;
      typename BlockExchange::TempStorage exchange;
    } temp;

    T thread_data[_4x4], l_result[1];
    T det;

    auto mul = [&] (T x, T y)      { return x * y; }; 
    auto fma = [&] (T x, T y, T z) { return mul(-x, y) + z; };

    BlockLoad(temp.load).Load(d_data, thread_data, Nmatrix * _4x4);
    __syncthreads();

    BlockExchange(temp.exchange).StripedToBlocked(thread_data);

    T a11 = thread_data[0];      T a21 = thread_data[4+0];
    T a12 = thread_data[1];      T a22 = thread_data[4+1];
    T a13 = thread_data[2];      T a23 = thread_data[4+2];
    T a14 = thread_data[3];      T a24 = thread_data[4+3];

    T a31 = thread_data[8+0];    T a41 = thread_data[12+0];
    T a32 = thread_data[8+1];    T a42 = thread_data[12+1];
    T a33 = thread_data[8+2];    T a43 = thread_data[12+2];
    T a34 = thread_data[8+3];    T a44 = thread_data[12+3];

    T a11_a22 = mul(a11, a22); T a11_a23 = mul(a11, a23); T a11_a24 = mul(a11, a24);
    T a12_a21 = mul(a12, a21); T a12_a23 = mul(a12, a23); T a12_a24 = mul(a12, a24);
    T a13_a21 = mul(a13, a21); T a13_a22 = mul(a13, a22); T a13_a24 = mul(a13, a24);
    T a14_a21 = mul(a14, a21); T a14_a22 = mul(a14, a22); T a14_a23 = mul(a14, a23);
    T a31_a42 = mul(a31, a42); T a31_a43 = mul(a31, a43); T a31_a44 = mul(a31, a44);
    T a32_a41 = mul(a32, a41); T a32_a43 = mul(a32, a43); T a32_a44 = mul(a32, a44);
    T a33_a41 = mul(a33, a41); T a33_a42 = mul(a33, a42); T a33_a44 = mul(a33, a44);
    T a34_a41 = mul(a34, a41); T a34_a42 = mul(a34, a42); T a34_a43 = mul(a34, a43);

    det = mul( a11_a22, a33_a44);
    det = fma( a11_a24, a32_a43, det);
    det = fma( a11_a23, a34_a42, det);
    det = fma(-a11_a24, a33_a42, det);
    det = fma(-a11_a22, a34_a43, det);
    det = fma(-a11_a23, a32_a44, det);
    det = fma(-a12_a21, a33_a44, det);
    det = fma(-a12_a23, a34_a41, det);
    det = fma(-a12_a24, a31_a43, det);
    det = fma( a12_a24, a33_a41, det);
    det = fma( a12_a21, a34_a43, det);
    det = fma( a12_a23, a31_a44, det);
    det = fma( a13_a21, a32_a44, det);
    det = fma( a13_a22, a34_a41, det);
    det = fma( a13_a24, a31_a42, det);
    det = fma(-a13_a24, a32_a41, det);
    det = fma(-a13_a21, a34_a42, det);
    det = fma(-a13_a22, a31_a44, det);
    det = fma(-a14_a21, a32_a43, det);
    det = fma(-a14_a22, a33_a41, det);
    det = fma(-a14_a23, a31_a42, det);
    det = fma( a14_a23, a32_a41, det);
    det = fma( a14_a21, a33_a42, det);

    l_result[0] = fma( a14_a22, a31_a43, det);
    BlockStore(temp.store).Store(d_result, l_result, Nmatrix); 

}

typedef cuda::std::complex<float> cfloat;
typedef cuda::std::complex<double> cdouble;

//template __global__ void det4x4kernel<double>(double *, double *, int);
//template __global__ void det4x4kernel<cdouble>(cdouble *, cdouble *, int);
template __global__ void det4x4kernel<float>(float *, float *, int);
//template __global__ void det4x4kernel<cfloat>(cfloat *, cfloat *, int);

int main()
{
    const int nmatrices = 32;

    // triadiagonal laplacian 4x4
    std::vector<float> iarray(_4x4, 0.f);
    iarray[0] = -4.f; iarray[1] =  2.f;
    iarray[4] =  1.f; iarray[5] = -4.f; iarray[6] = 1.f;
    iarray[9] =  1.f; iarray[10] = -4.f; iarray[11] = 1.f;
    iarray[14] =  2.f; iarray[15] = -4.f;

    float *d_array, *d_result;
    gpuErrchk( cudaMalloc(&d_array, nmatrices * _4x4 * sizeof(float)) );
    gpuErrchk( cudaMalloc(&d_result, nmatrices * sizeof(float)) );
    
    float *p = d_array;
    for(int i=0; i<nmatrices; i++) {
    	gpuErrchk( cudaMemcpy(p, &iarray[0], _4x4 * sizeof(float), cudaMemcpyHostToDevice) );
      p += _4x4;
      std::for_each(iarray.begin(), iarray.end(), [](float &x){ x*=1.005f;} );
    }
    
    int nblocks = (nmatrices / blocksz) + ((nmatrices % blocksz) ? 0 : 1);
    det4x4kernel<float><<<nblocks,blocksz>>>(d_array, d_result, nmatrices);
    gpuErrchk( cudaPeekAtLastError() );

    std::vector<float> result(nmatrices, -1.f);
    gpuErrchk( cudaMemcpy(&result[0], d_result, nmatrices * sizeof(float), cudaMemcpyDeviceToHost) );
    for(int i=0; i<nmatrices; i++) {
    	std::cout << result[i] << std::endl;
    }

    return 0;
}

Overwriting p4x4det.cu


In [25]:
!nvcc -arch=sm_75 -Xptxas="-v" -o p4x4det p4x4det.cu

p4x4det.cu(105): error: "det_a32" has already been declared in the current scope

p4x4det.cu(106): error: "det_a42" has already been declared in the current scope

p4x4det.cu(107): error: "det_a52" has already been declared in the current scope

3 errors detected in the compilation of "p4x4det.cu".


In [ ]:
!nvcc -arch=sm_75 -ptx p4x4det.cu
!cat p4x4det.ptx

Streaming output truncated to the last 5000 lines.
	selp.f32 	%f2945, %f2945, %f2285, %p1212;
	cvt.f64.f32 	%fd779, %f2944;
	abs.f64 	%fd780, %fd779;
	setp.le.f64 	%p1213, %fd780, 0d7FF0000000000000;
	mov.b32 	%r1672, %f2944;
	and.b32  	%r1673, %r1672, -2147483648;
	mov.b32 	%f2286, %r1673;
	selp.f32 	%f2944, %f2944, %f2286, %p1213;
	cvt.f64.f32 	%fd781, %f2943;
	abs.f64 	%fd782, %fd781;
	setp.le.f64 	%p1214, %fd782, 0d7FF0000000000000;
	mov.b32 	%r1674, %f2943;
	and.b32  	%r1675, %r1674, -2147483648;
	mov.b32 	%f2287, %r1675;
	selp.f32 	%f2943, %f2943, %f2287, %p1214;
	cvt.f64.f32 	%fd783, %f2942;
	abs.f64 	%fd784, %fd783;
	setp.le.f64 	%p1215, %fd784, 0d7FF0000000000000;
	mov.u16 	%rs611, 1;
	@%p1215 bra 	$L__BB3_506;

	mov.b32 	%r1676, %f2942;
	and.b32  	%r1677, %r1676, -2147483648;
	mov.b32 	%f2942, %r1677;

$L__BB3_506:
	setp.eq.s16 	%p1216, %rs611, 0;
	@%p1216 bra 	$L__BB3_508;

	mul.f32 	%f2288, %f2943, %f2945;
	mul.f32 	%f2289, %f2942, %f2944;
	sub.f32 	%f2290, %f2288, %f2289;


In [ ]:
!compute-sanitizer ./p4x4det

========= COMPUTE-SANITIZER
332
338.69
345.515
352.477
359.579
366.825
374.217
381.757
389.45
397.298
405.303
413.47
421.802
430.302
438.972
447.818
456.842
466.047
475.438
485.019
494.792
504.762
514.933
525.309
535.895
546.693
557.709
568.947
580.412
592.107
604.039
616.21
========= ERROR SUMMARY: 0 errors


In [ ]:
!nvprof ./p4x4det

==3309== NVPROF is profiling process 3309, command: ./p4x4det
332
338.69
345.515
352.477
359.579
366.825
374.217
381.757
389.45
397.298
405.303
413.47
421.802
430.302
438.972
447.818
456.842
466.047
475.438
485.019
494.792
504.762
514.933
525.309
535.895
546.693
557.709
568.947
580.412
592.107
604.039
616.21
==3309== Profiling application: ./p4x4det
==3309== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   86.33%  45.856us        32  1.4330us  1.3760us  1.9520us  [CUDA memcpy HtoD]
                    9.64%  5.1200us         1  5.1200us  5.1200us  5.1200us  void det4x4kernel<float>(float*, float*, int)
                    4.04%  2.1440us         1  2.1440us  2.1440us  2.1440us  [CUDA memcpy DtoH]
      API calls:   99.74%  194.36ms         2  97.180ms  5.6430us  194.35ms  cudaMalloc
                    0.16%  319.25us        33  9.6740us  4.7560us  29.117us  cudaMemcpy
                    0.06%  125.14us       101  1